# Single point calculation

To run a single point using aiida-mlip you need to define some inputs as AiiDA data types, to then pass them to the calculation.
First of all we need a structure on which to perform the calculations. It will be a NaCl structure that we define using ASE, or alternatively one can choose one of the structures in the folder `Structures`.
The input structure in aiida-mlip needs to be saved as a StructureData type:

In [1]:
from aiida import load_profile
load_profile()

Profile<uuid='60b17659a9844c4bbd3bef8de0a8f417' name='presto'>

In [2]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

#structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

Then we need to choose a model and architecture to be used for the calculation and save it as ModelData type, a specific data type of this plugin.
In this example we use MACE with a model that we download from this URL: "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model", and we save the file in the cache folder (default="~/.cache/mlips/"):


In [3]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

If we already have the model saved in some folder we can save it as:

In [4]:
# model = ModelData.from_local("/path/to/model", architecture="mace")

Another parameter that we need to define as AiiDA type is the code. Assuming the code is saved as `janus` in the `localhost` computer, the code info that are needed can be loaded as follow:


In [5]:
from aiida.orm import load_code
code = load_code("janus@localhost")

The other inputs can be set up as AiiDA Str. There is a default for every input except the structure and code. This is a list of possible inputs:

In [6]:
from aiida.orm import Dict, Str

inputs = {
        "code": code,
        "model": model,
        "struct": structure,
        "arch": Str(model.architecture),
        "device": Str("cpu"),
        "calc_kwargs": Dict({"dispersion": True}),
        "metadata": {"options": {"resources": {"num_machines": 1}}},
    }

It's worth noting that the architecture is already defined within the model, accessible through the architecture property in the ModelData. Even if not explicitly provided as input, it will be automatically retrieved from the model.

The calculation must be set:

In [7]:
from aiida.plugins import CalculationFactory
singlepointCalc = CalculationFactory("mlip.sp")

In this case, since we are running a single point calculation the entry point for the calculation is `mlip.sp`.
Finally, run the calculation:


In [8]:
from aiida.engine import run_get_node
result, node = run_get_node(singlepointCalc, inputs)

12/08/2025 12:22:22 PM <26765> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.12.1 is not supported and will cause unexpected problems!
12/08/2025 12:22:22 PM <26765> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
12/08/2025 12:22:22 PM <26765> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.


defining outputnode


`result` is a dictionary of the available results obtained from the calculation, while node contains the info on the node where the calculation is run:


In [9]:
print(result)
print(node)

{'remote_folder': <RemoteData: uuid: a9965df1-5b4f-4100-90b1-9a23d5ae2e68 (pk: 3164)>, 'retrieved': <FolderData: uuid: 70fb9e22-bb08-42e6-ac81-fb3ceeb9b7f7 (pk: 3165)>, 'log_output': <SinglefileData: uuid: 3586287e-c2c2-42bf-9f54-4bdd9751500a (pk: 3166)>, 'std_output': <SinglefileData: uuid: 75fb53ac-b5a0-4448-9d3b-e05bef182cb0 (pk: 3167)>, 'xyz_output': <SinglefileData: uuid: 4ff3023e-5591-409f-a876-0d584f3075a4 (pk: 3168)>, 'results_dict': <Dict: uuid: b59d8cec-5a5e-4761-b65c-9544d093977a (pk: 3169)>}
uuid: def1995f-6da1-4a64-a9d8-dea7fe2f01b5 (pk: 3163) (aiida.calculations:mlip.sp)


We can check if the calculation finished with errors. If everything worked the exit code should be 0

In [ ]:
if node.is_finished_ok:
     print(f"Calculation is finished without errors with exit status {node.exit_status}")
else:
     print(f"Some errors occurred with exit status {node.exit_status}") 

Caculation is finished without errors with exit status 0


If more information are needed on specific outputs they can be called like:

In [11]:
print(result["results_dict"].get_dict())

{'numbers': [11, 17], 'positions': [[0.0, 0.0, 0.0], [2.815, 0.0, 0.0]], 'masses': [22.98976928, 35.45], 'mace_mp_d3_forces': [[0.0, -0.0, 0.0], [-0.0, 0.0, -0.0]], 'cell': [[0.0, 2.815, 2.815], [2.815, 0.0, 2.815], [2.815, 2.815, 0.0]], 'pbc': [True, True, True], 'info': {'units': {'energy': 'eV', 'forces': 'ev/Ang', 'stress': 'ev/Ang^3'}, 'emissions': 8.2355259081593e-07, 'model': 'mlff.model', 'arch': 'mace_mp_d3', 'mace_mp_d3_energy': -7.1874941952815, 'mace_mp_d3_stress': [0.0042117876302604, 0.0042117876302604, 0.0042117876302604, -2.3330084564496e-19, 1.1681803864766e-18, -4.5798633012734e-18], 'system_name': 'aiida'}}


Let's say we want the energy we can access it via the result or the node variable.
(when this is run in the terminal the auto-completion should help, but the idea is that the results_dict is one of the outputs which contains the main info on the calculation)

In [12]:
print(f"Energy: {result['results_dict'].get_dict()['info']['mace_mp_d3_energy']}")
print(f"Energy: {node.outputs.results_dict.get_dict()['info']['mace_mp_d3_energy']}")
print(node.outputs.results_dict.get_dict()['cell'][0][1])

Energy: -7.1874941952815
Energy: -7.1874941952815
2.815


Through the command line we can see the processes that are run

In [13]:
! verdi process list -a

  PK  Created    Process label                        ♻    Process State     Process status
----  ---------  -----------------------------------  ---  ----------------  ---------------------------------------------------------------------------------------------------
 114  67D ago    Singlepoint                               ⏹ Finished [0]
 132  67D ago    GeomOpt                                   ⏹ Finished [0]
 153  67D ago    GeomOpt                                   ⏹ Finished [0]
 164  67D ago    prepare_struct_inputs                     ⏹ Finished [0]
 170  67D ago    Singlepoint                               ⏹ Finished [0]
 188  67D ago    GeomOpt                                   ⏹ Finished [0]
 199  67D ago    prepare_struct_inputs                     ⏹ Finished [0]
 205  67D ago    Singlepoint                               ⏹ Finished [0]
 223  66D ago    GeomOpt                                   ⏹ Finished [0]
 234  66D ago    prepare_struct_inputs                     ⏹ Fini

And see the results we are interested in. Substitute the number with the PK number of your calculation

In [16]:
! verdi calcjob res 3163

defining outputnode
{
    "cell": [
        [
            0.0,
            2.815,
            2.815
        ],
        [
            2.815,
            0.0,
            2.815
        ],
        [
            2.815,
            2.815,
            0.0
        ]
    ],
    "info": {
        "arch": "mace_mp_d3",
        "emissions": 8.2355259081593e-07,
        "mace_mp_d3_energy": -7.1874941952815,
        "mace_mp_d3_stress": [
            0.0042117876302604,
            0.0042117876302604,
            0.0042117876302604,
            -2.3330084564496e-19,
            1.1681803864766e-18,
            -4.5798633012734e-18
        ],
        "model": "mlff.model",
        "system_name": "aiida",
        "units": {
            "energy": "eV",
            "forces": "ev/Ang",
            "stress": "ev/Ang^3"
        }
    },
    "mace_mp_d3_forces": [
        [
            0.0,
            -0.0,
            0.0
        ],
        [
            -0.0,
            0.0,
            -0.0
        ]

We can also see the inputs and outputs of the calculation

In [17]:
! verdi node show 3163

Property     Value
-----------  ------------------------------------
type         Singlepoint
state        Finished [0]
pk           3163
uuid         def1995f-6da1-4a64-a9d8-dea7fe2f01b5
label
description
ctime        2025-12-08 12:22:22.220454+00:00
mtime        2025-12-08 12:22:35.885722+00:00
computer     [1] localhost

Inputs          PK  Type
------------  ----  -------------
arch          3157  Str
calc_kwargs   3159  Dict
code             1  InstalledCode
device        3158  Str
log_filename  3160  Str
model          106  ModelData
out           3162  Str
struct        3156  StructureData
summary       3161  Str

Outputs          PK  Type
-------------  ----  --------------
log_output     3166  SinglefileData
remote_folder  3164  RemoteData
results_dict   3169  Dict
retrieved      3165  FolderData
std_output     3167  SinglefileData
xyz_output     3168  SinglefileData
